In [1]:
import numpy as np
from pathlib import Path
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from crop_face import convert_face_tensor_to_numpy_array, prepare_single_face_model

In [2]:
save_state_path = Path("./custom_image_labelling_save_state")

In [3]:
labels = np.load(save_state_path / "labels.npy")
marked_for_deletion = np.load(save_state_path / "marked_for_deletion.npy")
marked_for_manual = np.load(save_state_path / "marked_for_manual.npy")

In [4]:
(labels[:,0] == 1).sum()

12

In [5]:
(marked_for_deletion == 1).sum()

202

In [6]:
(marked_for_manual == 1).sum()

17

In [7]:
images_pth = Path("./data/images")

In [8]:
from utils import load_images_recursively

images, _ = load_images_recursively(images_pth)

In [9]:
idx_not_marked_for_deletion = np.where(marked_for_deletion == 0)[0]
idx_not_marked_for_manual = np.where(marked_for_manual == 0)[0]

In [10]:
len(idx_not_marked_for_deletion)

1310

In [11]:
to_keep = []

for i in range(len(images)):
    if i in idx_not_marked_for_deletion and i in idx_not_marked_for_manual:
        to_keep.append(i)
        
to_keep = np.array(to_keep)

In [12]:
to_keep

array([   2,    3,    5, ..., 1509, 1510, 1511])

In [13]:
filtered_images =[images[i] for i in to_keep]

In [14]:
filtered_labels = [labels[i] for i in to_keep]

In [15]:
len(images)

1512

In [16]:
len(filtered_images)

1293

In [17]:
def crop_and_transform(img):
    return convert_face_tensor_to_numpy_array(cropping_model(right_half))

In [18]:
new_crops_path = Path('./data/new_crops')
before_crops_path = new_crops_path / 'before'
after_crops_path = new_crops_path / 'after'

In [19]:
images_and_labels = zip(filtered_images, filtered_labels)

In [20]:
from crop_face import prepare_single_face_model

In [22]:
more_manual = np.zeros([len(filtered_images)])

for i, (img_path, labels) in tqdm(enumerate(images_and_labels), total=len(filtered_images)):
    try:
        im = np.array(Image.open(img_path))
        height = im.shape[0]
        width = im.shape[1]
        width_cutoff = width // 2
        left_half = im[:,0:width_cutoff,:]
        right_half = im[:,width_cutoff:width,:]

        path_parts = img_path.parts
        img_name = path_parts[-2] + '_' + path_parts[-1]
        before_img_path = before_crops_path / img_name
        after_img_path = after_crops_path / img_name


        before = left_half if labels[0] == 0 else right_half
        after = left_half if labels[0] == 1 else right_half

        cropping_model = prepare_single_face_model(margin=30)
        crop_and_transform = lambda i: convert_face_tensor_to_numpy_array(cropping_model(i))

        try:
            cropped_before = crop_and_transform(before)
            Image.fromarray(cropped_before).convert('RGB').save(before_img_path)
        except Exception as e:
            print(e)
            print(f'Before could not be extracted from {img_path}')
            more_manual[i] = 1
            Image.fromarray(left_half).convert('RGB').save(before_img_path)

        try:
            cropped_after = crop_and_transform(after)
            Image.fromarray(cropped_after).convert('RGB').save(after_img_path)
        except Exception as e:
            print(e)
            print(f'After could not be extracted from {img_path}')
            more_manual[i] = 1
            Image.fromarray(right_half).convert('RGB').save(after_img_path)
    except Exception as e:
        print(e)
        continue
    

  1%|▌                                        | 19/1293 [00:02<02:07, 10.00it/s]

'NoneType' object has no attribute 'permute'
After could not be extracted from data/images/beard before after 2/19.jpg


  3%|█                                        | 33/1293 [00:03<01:53, 11.09it/s]

Given groups=1, weight of size [10, 3, 3, 3], expected input[1, 4, 229, 172] to have 3 channels, but got 4 channels instead
Before could not be extracted from data/images/beard before after 2/103.jpg
Given groups=1, weight of size [10, 3, 3, 3], expected input[1, 4, 229, 172] to have 3 channels, but got 4 channels instead
After could not be extracted from data/images/beard before after 2/103.jpg


  7%|██▊                                      | 89/1293 [00:09<01:57, 10.28it/s]

...

 44%|█████████████████▊                      | 575/1293 [00:56<01:15,  9.47it/s]

Given groups=1, weight of size [10, 3, 3, 3], expected input[1, 4, 271, 245] to have 3 channels, but got 4 channels instead
Before could not be extracted from data/images/beard before after 3/107.jpg
Given groups=1, weight of size [10, 3, 3, 3], expected input[1, 4, 271, 245] to have 3 channels, but got 4 channels instead
After could not be extracted from data/images/beard before after 3/107.jpg


 45%|██████████████████                      | 583/1293 [00:57<01:26,  8.19it/s]

'NoneType' object has no attribute 'permute'
After could not be extracted from data/images/hiardoo before after men 6/3.jpg


 47%|██████████████████▋                     | 603/1293 [00:58<01:07, 10.25it/s]
